If on google colab, you will need to change the hardware accelerator to GPU under Runtime > Change Runtime Type.

Then clone and install the cbmos github repository via

`!git clone https://github.com/somathias/cbmos.git`

`%cd cbmos`

`!pip install -e .`


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import cbmos
import cbmos.force_functions as ff
import cbmos.solvers.euler_forward as ef
import cbmos.cell as cl

In [ ]:
# assuming GPU backend is available and accessible through CuPy
import cupy as cp

Define two models both using the forward Euler method as a solver - one using NumPy as a hpc_backend and the other using CuPy

In [ ]:
dim = 2 # let's have a two-dimensional model
cbmmodel_numpy = cbmos.CBMModel(ff.Linear(), ef.solve_ivp, dim, hpc_backend=np)
cbmmodel_cupy = cbmos.CBMModel(ff.Linear(), ef.solve_ivp, dim, hpc_backend=cp)

Let's generate a large set of random numbers...

In [ ]:
seed = 845
np.random.seed(seed) # set seed for reproducibility if necessary
N = 10000
y_np = np.random.uniform(-10, 10, size=(N, dim))

...which we can use as initial coordinates of our (non-proliferating) cell population.

In [ ]:
cell_list = []
N = 1000
for i in range(N):
    cell_list.append(cl.Cell(i, y_np[i, :]))

Now time the models on the mechanical relaxation of our random cell population.

In [ ]:
t_data = np.linspace(0, 1, 101)

In [ ]:
%time history = cbmmodel_numpy.simulate(cell_list, t_data, {}, {"dt": 0.01}, seed=seed)

In [ ]:
%time history = cbmmodel_cupy.simulate(cell_list, t_data, {}, {"dt": 0.01}, seed=seed)